In [96]:
import numpy as np
import math
import numpy.matlib

In [97]:
x_train = np.array([0.09291784,0.46809093,0.93089486,0.67612654,0.73441752,0.86847339,\
                   0.49873225,0.51083168,0.18343972,0.99380898,0.27840809,0.38028817,\
                   0.12055708,0.56715537,0.92005746,0.77072270,0.85278176,0.05315950,\
                   0.87168699,0.58858043])
y_train = np.array([-0.25934537,0.18195445,0.651270150,0.13921448,0.09366691,0.30567674,\
                    0.372291170,0.20716968,-0.08131792,0.51187806,0.16943738,0.3994327,\
                    0.019062570,0.55820410,0.452564960,-0.1183121,0.02957665,-1.24354444, \
                    0.248038840,0.26824970])
y_train_binary = np.array([0,1,1,0,0,1,\
                    1,0,0,1,0,1,\
                    0,1,1,0,1,0, \
                    1,1])
y_train_multiclass = np.array([2,0,1,2,1,0,\
                    0,2,2,0,2,0,\
                    2,0,1,2,1,2, \
                    1,0])

Define the Rectified Linear Unit (ReLU) function 

In [98]:
def ReLU(preactivation):
    return np.clip(preactivation,a_min=0,a_max=1)


In [99]:
# Sigmoid function that maps [-infty,infty] to [0,1]
def sigmoid(model_out):
  sig_model_out = 1/(1+np.exp(-model_out))
  return sig_model_out

In [100]:
# Softmax function that maps a vector of arbitrary values to a vector of values that are positive and sum to one.
def softmax(model_out):
  # Compute the exponential of the model outputs
  exp_model_out = np.exp(model_out)
  # Compute the sum of the exponentials (denominator of equation 5.22)
  sum_exp_model_out = np.sum(exp_model_out, axis =0)
  # Normalize the exponentials (np.matlib.repmat might be useful here)
  softmax_model_out = exp_model_out/np.matlib.repmat(sum_exp_model_out, exp_model_out.shape[0], 1)

  return softmax_model_out

Define a shallow neural network with two inputs, two outputs, and three hidden units

In [101]:
def shallow_2_2_3(x1,x2, activation_fn, phi_10, phi_11, phi_12, phi_13, phi_20, phi_21, phi_22, phi_23, theta_10, theta_11, theta_12, theta_20, theta_21, theta_22, theta_30, theta_31, theta_32):
    h1 = activation_fn(theta_10 + theta_11*x1 + theta_12*x2)
    h2 = activation_fn(theta_20 + theta_21*x1 + theta_22*x2)
    h3 = activation_fn(theta_30 + theta_31*x1 + theta_32*x2)

    y1 = phi_10 + phi_11*h1 + phi_12*h2 + phi_13*h3
    y2 = phi_20 + phi_21*h1 + phi_22*h2 + phi_23*h3

    

Shallow Neural Network in matrix form

In [102]:
def shallow_1_1_1(x, beta_0, omega_0, beta_1, omaga_1):
    n_data = x.size
    x = np.reshape(x,(1,n_data))
    h1 = ReLU(np.matmul(beta_0,np.ones((1,n_data))) + np.matmul(omega_0,x))
    y = np.matmul(beta_1,np.ones((1,n_data))) + np.matmul(omega_1,h1)
    return y

compute the maximum possible number of linear regions

In [103]:
def number_regions(Di, D):
  # TODO -- implement Zaslavsky's formula
  # You will need to use math.factorial() https://www.geeksforgeeks.org/factorial-in-python/
  # Replace this code
  N = 0
  for j in range(0, Di+1):
    N += factorial(D)/(factorial(D - j)* factorial(j))

  return N

Define a deep neural network with, one input, one output, two hidden layers and three hidden units

In [104]:
# define sizes
D_i=4; D_1=5; D_2=2; D_3=1; D_o=1
# We'll choose the inputs and parameters of this network randomly using np.random.normal
# For example, we'll set the input using
n_data = 4;
x = np.random.normal(size=(D_i, n_data))
# TODO initialize the parameters randomly with the correct sizes
# Replace the lines below
beta_0 = np.random.normal(size=(D_1,1))
Omega_0 = np.random.normal(size=(D_1,D_i))
beta_1 = np.random.normal(size=(D_2,1))
Omega_1 = np.random.normal(size=(D_2,D_1))
beta_2 = np.random.normal(size=(D_3,1))
Omega_2 = np.random.normal(size=(D_3,D_2))
beta_3 = np.random.normal(size=(D_o,1))
Omega_3 = np.random.normal(size=(D_o,D_3))


# If you set the parameters to the correct sizes, the following code will run
h1 = ReLU(np.matmul(beta_0,np.ones((1,n_data))) + np.matmul(Omega_0,x));
h2 = ReLU(np.matmul(beta_1,np.ones((1,n_data))) + np.matmul(Omega_1,h1));
h3 = ReLU(np.matmul(beta_2,np.ones((1,n_data))) + np.matmul(Omega_2,h2));
y = np.matmul(beta_3,np.ones((1,n_data))) + np.matmul(Omega_3,h3)

Convert neural network in matrix form

Loss function

In [105]:
def compute_sum_of_squares(y_train, y_pred):
    sum_of_squares = np.sum((y_train-y_pred) * (y_train-y_pred))
    return sum_of_squares

In [106]:
beta_0 = np.zeros((3,1));  # formerly theta_x0
omega_0 = np.zeros((3,1)); # formerly theta_x1
beta_1 = np.zeros((1,1));  # formerly phi_0
omega_1 = np.zeros((1,3)); # formerly phi_x

beta_0[0,0] = 0.3; beta_0[1,0] = -1.0; beta_0[2,0] = -0.5
omega_0[0,0] = -1.0; omega_0[1,0] = 1.8; omega_0[2,0] = 0.65
beta_1[0,0] = 0.1;
omega_1[0,0] = -2.0; omega_1[0,1] = -1.0; omega_1[0,2] = 7.0

Univeriate Regression

In [107]:
# Return probability under normal distribution for input x
def normal_distribution(y, mu, sigma):
    prob = (1/np.sqrt(2*math.pi*sigma*sigma))* np.exp(-((y-mu) * (y-mu)/(2*sigma*sigma)))
    return prob

In [108]:
def compute_likelihood(y_train, mu, sigma):
    likelihood = np.prod(normal_distribution(y_train, mu, sigma))
    return likelihood
def compute_negative_log_likelihood(y_train, mu, sigma):
    nll = -1 * np.sum(np.log(normal_distribution(y_train, mu, sigma)))
    return nll

In [109]:
mu_pred = shallow_1_1_1(x_train, beta_0, omega_0, beta_1, omega_1)
sigma = 0.2
likelihood = compute_likelihood(y_train, mu_pred, sigma)
nll = compute_negative_log_likelihood(y_train, mu_pred, sigma)
sum_of_squares = compute_sum_of_squares(y_train, mu_pred)

Maximum of the likelihood fn is at the same position as the minimum negative log likelihood and the least squares solutions

Binary Cross Entropy Loss

In [110]:
# Return probability under Bernoulli distribution for input x
def bernoulli_distribution(y, lambda_param):
    prob = np.power(1-lambda_param,(1-y)) * np.power(lambda_param,y)
    return prob

In [111]:
# Return the likelihood of all of the data under the model
def compute_likelihood(y_train, lambda_param):
  prob = bernoulli_distribution(y_train, lambda_param)
  likelihood = np.prod(prob)
  return likelihood

# Return the negative log likelihood of the data under the model
def compute_negative_log_likelihood(y_train, lambda_param):
  nll = -1 * np.sum(np.log(bernoulli_distribution(y_train, lambda_param)))
  return nll

In [112]:
model_out = shallow_1_1_1(x_train, beta_0, omega_0, beta_1, omega_1)
lambda_train = sigmoid(model_out)
likelihood = compute_likelihood(y_train_binary, lambda_train)
nll = compute_negative_log_likelihood(y_train_binary, lambda_train)
sum_of_squares = compute_sum_of_squares(y_train_binary, model_out)

Multiclass Cross Entropy Loss

In [113]:
# Return probability under Categorical distribution for input x
# Just take value from row k of lambda param where y =k,
def categorical_distribution(y, lambda_param):
    return np.array([lambda_param[row, i] for i, row in enumerate (y)])

In [114]:
# Return the likelihood of all of the data under the model
def compute_likelihood(y_train, lambda_param):
  likelihood = np.prod(categorical_distribution(y_train, lambda_param))
  return likelihood

# Return the negative log likelihood of the data under the model
def compute_negative_log_likelihood(y_train, lambda_param):
  nll = -1 * np.sum(np.log(categorical_distribution(y_train, lambda_param)))
  return nll

model_out = shallow_1_1_1(x_train, beta_0, omega_0, beta_1, omega_1)
# Pass the outputs through the softmax function
lambda_train = softmax(model_out)
likelihood = compute_likelihood(y_train_multiclass, lambda_train)
nll = compute_negative_log_likelihood(y_train_multiclass, lambda_train)

IndexError: index 2 is out of bounds for axis 0 with size 1